<a href="https://colab.research.google.com/github/itayle/MeshRunner/blob/main/MeshRunner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Download Shrec
import os
os.chdir("/content")
!rm -r MeshRunner
!git clone https://github.com/itayle/MeshRunner
!wget https://cgm.technion.ac.il/Computer-Graphics-Multimedia/Software/MeshWalker/mesh_walker_data/datasets_processed/shrec11.tar.gz -P MeshRunner/datasets_processed
!tar -xzvf MeshRunner/datasets_processed/shrec11.tar.gz --directory MeshRunner/datasets_processed


# install libraries
Need to restart notebook after numpy installation

In [ ]:
!pip install numpy==1.19.5
!pip install tensorflow==2.2.0
!pip install tensorflow-addons==0.8.3
!pip install open3d==0.8.0.0
!pip install easydict
!pip install h5py
!pip install ipython
!pip install matplotlib==3.*
!pip install networkx==2.*
!pip install psutil
!pip install scikit-learn==0.22.*
!pip install scipy==1.*
!pip install tqdm
!pip install trimesh==3.*
#!pip install pyvista==0.24.*
!pip install pydot
!pip install graphviz
!pip install opencv-python==4.*

!pip install pyvista ipyvtklink trimesh
!pip install rtree
# To avoid "ModuleNotFoundError: No module named 'vtkOpenGLKitPython' " when importing vtk
# https://stackoverflow.com/q/32389599
# https://askubuntu.com/q/629692
!sudo apt-get update
!sudo apt install python-qt4 libgl1-mesa-glx
!sudo apt-get install xvfb
!export DISPLAY=:99.0
!export PYVISTA_OFF_SCREEN=true
!export PYVISTA_USE_IPYVTK=true

# Training script
Please choose training args (attention, type of walk , ..)

In [ ]:
attention=True
encodeJumps = True
saliency=False
walk_name = "regular" # Options: ["regular", "skip", "walk_with_jumps","order"]

In [ ]:
import os
print(os.getcwd())
os.chdir("/content/MeshRunner/")
print(os.getcwd())

import numpy as np
import utils
import importlib
import tensorflow as tf
import train_val
# importlib.reload(dataset)
# importlib.reload(train_val)
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

utils.config_gpu()
job = "shrec11"
job_part = "10-10_A"

run_name = "seed_" + str(seed)
run_name += "att_" + str(attention)

train_val.run_one_job(job=job,
                        job_part=job_part,
                        optim="adam",
                        lr=1e-4,
                        num_of_epochs_for_validtion=200,
                        attention=attention,
                        saliency=saliency,
                        encodeJumps=encodeJumps,
                        walk_name = walk_name,
                        custom_name=run_name)

# Evaluation script
please choose your run directory (in /content/MeshRunner/runs/) and evaluate your model after training for at least 1115 epochs (in order for a .keras file to be created)

In [ ]:
dir = "/runs/0001-19.02.2022..18.28__shrec11_10-10_A_seed_0att_False"


In [ ]:
importlib.reload(train_val)
np.random.seed(0)
tf.random.set_seed(0)
params = train_val.get_params(job, job_part)
seq_len_list = [25, 50, 100]
n_walks_list = [1, 4, 16, 32]
for seq_len in seq_len_list:
    print("---------seq_len:", seq_len, "---------")
    for n_walks in n_walks_list:
        acc, _ = train_val.calc_accuracy_test(logdir=dir, **params.full_accuracy_test, n_walks_per_model=n_walks,
                                            iter2use='00010008.keras',seq_len=seq_len)
        print("n_walks:", n_walks, "acc", acc[0])